In [1]:
import os, sys
# PWD = os.getenv('PWD')
PWD = os.path.abspath('.')

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [2]:
init_django('django_trip_webserver') #project name

In [3]:
from board.models import Trip, TripComment
from main.models import TripCategory, AreaL, AreaM
from user.models import User

In [4]:
import pandas as pd
import json
import re
df = pd.read_csv(r'C:\Users\User\Desktop\깃_웹서버\skku-webproject-git\asdfasdfasdf\df_detail.csv', sep=',')
rdf = pd.read_csv(r'C:\Users\User\Desktop\깃_웹서버\skku-webproject-git\asdfasdfasdf\df_rev.csv', sep=',')


In [5]:
df.drop(df[df['place_category_group'] == '기타'].index, axis=0, inplace=True)
df = df.reset_index()
df = df.drop(columns = 'index')
df

,trip_name,trip_category_detail,trip_address,trip_times,place_page,trip_phone,info_content,place_category_group
0,신라스테이 광화문,신라스테이,서울 종로구 삼봉로 71 G타워 (우)03150,-,www.shillastay.com/gwanghwamun,02-6060-9000,none,숙소
1,JW메리어트동대문스퀘어서울,메리어트호텔,서울 종로구 청계천로 279 (우)03198,-,www.instagram.com/jwmarriott_dongdaemun,02-2276-3000,none,숙소
2,이비스앰배서더호텔 인사동,앰배서더호텔,서울 종로구 삼일대로30길 31 (우)03132,-,ibis.ambatel.com/insadong,02-6730-1101,주차,숙소
3,노블관광호텔,베니키아 호텔,서울 종로구 율곡로6길 13-3 (우)03131,-,www.noblehotel.co.kr,02-742-4025,none,숙소
4,포시즌스호텔 서울,호텔,서울 종로구 새문안로 97 (우)03183,매일 00:00 ~ 24:00,www.fourseasons.com/kr/seoul,02-6388-5000,주차,숙소
...,...,...,...,...,...,...,...,...
3835,호스텔코리아 창덕궁점,호스텔,서울 종로구 돈화문로 85 호스텔코리아,-,http://cdg.hostelkorea.com/,070-4705-1900,Hostel Korea 창덕궁 'SAFE STAY away from HOME' 호스...,숙소
3836,인사동미니호텔,모텔,서울 종로구 인사동14길 26 인사동미니호텔,-,-,02-733-1355,인사동 미니호텔은 안국역에서 150m 거리에 있는 미니호텔 게스트하우스 입니다. ...,숙소
3837,숭례문,기념물,서울 중구 세종대로 40,"['화: 09:00 - 18:00', '수: 09:00 - 18:00', '목: 0...",-,02-779-8547,-,관광
3838,준오호텔,호텔,서울 종로구 인사동16길 6 2층,-,http://site.onda.me/133404,-,서울의 중심 종로 인사동에 위치한 종로 준오 호텔입니다 코로나로 잠시 휴식 기간...,숙소


In [6]:
rdf = rdf.rename(columns={'place_name':'trip_name'})
rdf['comment_star'] = rdf['comment_star'].replace('-', '5.0') # 평점이 없는 것 임의로 5점 줌
rdf['comment_star'] = rdf['comment_star'].astype(float).astype(int)

In [13]:
rdf['comment_content'] = rdf['comment_content'].replace('-', 'None')
rdf['comment_time']

0        2023.10.04.
1        2023.10.03.
2        2023.10.01.
3        2023.10.01.
4        2023.09.27.
            ...     
42136           4년 전
42137           2년 전
42138           2년 전
42139           1년 전
42140           1년 전
Name: comment_time, Length: 42141, dtype: object

In [8]:
area_large = AreaL(
    area_l_name='서울'
)

area_large.save()

In [9]:
area_medium = AreaM(
    area_l_id = area_large,
    area_m_name = '종로'
)

area_medium.save()

In [10]:
trip_category_dic = {'명소':1, '관광':2, '쇼핑':3, '식사':4, '카페':5, '숙소':6}
for i in range(len(df)):
    category_key = df['place_category_group'][i]

    Trip.objects.create(
        trip_category_id = TripCategory.objects.get(id=trip_category_dic[category_key]),
        trip_category_detail = df['trip_category_detail'][i],
        trip_name = df['trip_name'][i],
        trip_address = df['trip_address'][i],
        area_m_id = AreaM.objects.get(id=1),
        trip_time = df['trip_times'][i],
        trip_phone = df['trip_phone'][i],
        trip_homepage = df['place_page'][i],
        is_deleted = False
    )

In [11]:
from user.models import User

for u in range(len(rdf)):
    User.objects.create(
        user_name = rdf['comment_user'][u],
        user_password = 1234,
        trip_category_1_id = TripCategory.objects.get(id=1),
        trip_category_2_id = TripCategory.objects.get(id=2),
        trip_category_3_id = TripCategory.objects.get(id=3),
        is_deleted = False
    )

In [12]:
from board.models import Trip, TripComment
from user.models import User

for t in range(len(rdf)):
    trip_instance = Trip.objects.filter(trip_name=rdf['trip_name'][t]).first()
    user_instance = User.objects.filter(user_name=rdf['comment_user'][t]).first()
    TripComment.objects.create(
        user_id=user_instance,
        trip_id=trip_instance,
        trip_comment_content=rdf['comment_content'][t],
        trip_comment_time=rdf['comment_time'][t],
        trip_comment_star=rdf['comment_star'][t],
        is_deleted=False
    )